In [18]:
import pandas as pd
from biopandas.pdb import PandasPdb

# Read in the PDB file using Biopandas
ppdb = PandasPdb()
ppdb.read_pdb('5xra.pdb')

# Remove any 'ATOM' records whose residue number is greater than 1000 ie T4 lysozyme
ppdb.df['ATOM'] = ppdb.df['ATOM'][ppdb.df['ATOM']['residue_number'] <= 1000]

# Save the modified dataframe to another PDB file
ppdb.to_pdb(path='output.pdb', records=['ATOM'])

In [ ]:
import Bio.PDB
from io import StringIO
from Bio import SeqIO
from Bio.PDB import PDBParser, PDBIO
from Bio.SeqUtils import seq3

uniprot_id = 'P21554'
parser = Bio.PDB.PDBParser()
structure = parser.get_structure('protein', 'output.pdb')
url = f'https://www.uniprot.org/uniprot/{uniprot_id}.fasta'
response = requests.get(url)
wildtype_seq = SeqIO.read(StringIO(response.text), 'fasta').seq

# Loop through the residues and compare to the wild-type sequence
for residue in structure.get_residues():
    res_num = residue.id[1]
    wt_res = wildtype_seq[res_num-1]
    pdb_res = seq3(residue.resname)
    if wt_res != pdb_res:
        # Revert the mutation by changing the residue identity (in three-letter code) to the wild-type residue
        residue.resname = seq3(wt_res)

# Save the modified PDB file
io = PDBIO()
io.set_structure(structure)
io.save("protein_wt.pdb")

In [ ]:
# Load the PDB file
ppdb = PandasPdb()
ppdb.read_pdb("protein_wt.pdb")

# Isolate CA atoms
ppdb.df['ATOM'] = ppdb.df["ATOM"][ppdb.df["ATOM"]["atom_name"] == "CA"]

# Save the modified dataframe to another PDB file
ppdb.to_pdb(path='CA_output.pdb', records=['ATOM'])

In [29]:
# Load the PDB file
ppdb = PandasPdb()
ppdb.read_pdb("protein_wt.pdb")

# Isolate backbone atoms
ppdb.df['ATOM'] = ppdb.df['ATOM'][(ppdb.df['ATOM']['atom_name'] == 'C') | 
                            (ppdb.df['ATOM']['atom_name'] == 'O') | 
                            (ppdb.df['ATOM']['atom_name'] == 'N') | 
                            (ppdb.df['ATOM']['atom_name'] == 'CA')]

# Save the modified dataframe to another PDB file
ppdb.to_pdb(path='Backbone.pdb', records=['ATOM'])